In [2]:
import requests
from bs4 import BeautifulSoup
import time
import json
import polars as pl
from tqdm import tqdm
import polars.selectors as cs

In [ ]:
# Control whether to scrape for data or read from file
SCRAPE_FINALS_YEARS= False
SCRAPE_TEAM_ROSTERS = False
SCRAPE_PLAYER_TEAMS = False

In [3]:
"""
Function to scrape a webpage and return a BeautifulSoup object.
:param url: The URL of the webpage to scrape.
:param session: Optional requests.Session object for persistent connections.
:return: BeautifulSoup object containing the parsed HTML of the page.
"""
def scrape(url, session=None):
    # Request the page
    response = requests.get(url) if session is None else session.get(url)
    # Parse with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    time.sleep(2)  # Sleep to avoid rate limiting
    return soup

In [4]:
"""
Function to get the list of teams a player has played for from a given URL.
:param url: The URL of the player's stats page.
:return: A list of unique team names the player has played for.
"""
def get_player_teams(url):
    
    # Retry the request until it succeeds
    retry_count = 0
    while retry_count < 5:
        try:
            # Request the page and parse it with BeautifulSoup
            soup = scrape(url)
            
            # Find the table with player stats
            table = soup.find('table', {'id': 'per_game_stats'})
            time.sleep(3)  # Sleep to avoid rate limiting
            # If no exception occurs, break the loop
            break
        except Exception as e:
            print(f"Error occurred: {e}. Retrying...")
            # If an exception occurs, retry after a delay
            retry_count += 1
            time.sleep(60)
    # If the table is not found, raise an exception
    if table is None:
        raise Exception("Table not found on the page.")
    
    teams = []
    # Iterate through the rows of the table in tbody
    for row in table.tbody.find_all('tr'):
        # Find the team name in the row
        team = row.find('td', {'data-stat': 'team_name_abbr'})
        # If the team name is found, add it to the list
        if team:
            teams.append(team.text.strip())
    
    # Return a list of unique team names
    return list(set(teams))
    

In [5]:
"""
Function to get the teams that played in the finals from a given URL.
:param url: The URL of the playoffs page.
:param session: Optional requests.Session object for persistent connections.
:return: A tuple containing the URLs of the two teams that played in the finals.
"""
def get_finals_teams(url, session):
    # Request the page and parse it with BeautifulSoup
    soup = scrape(url, session)
    # Find the playoff table
    playoff_table = soup.find('table', {'id': 'all_playoffs'}).tbody
    # Go through the rows to find the two teams in the finals
    for row in playoff_table.find_all('tr'):
        # Check if the row contains 'Finals'
        for cell in row.find_all('td'):
            if cell.text.strip() == 'Finals':
                # If found, extract the team names
                # The team names are in the second column of the row
                teams = row.find_all('td')[1].find_all('a')
                team1 = teams[0]['href']
                team2 = teams[1]['href']
                return team1, team2

In [16]:
"""
Function to get the teams that played in the finals for each year.
:return: A dictionary mapping each year to a tuple of team URLs that played in the finals.
"""
def get_playoff_series():
    # Start a new session
    session = requests.Session()
    # Request the page and parse it with BeautifulSoup
    all_playoffs_url = "https://www.basketball-reference.com/playoffs/"
    soup = scrape(all_playoffs_url, session)
    # Find the table with playoff series
    table = soup.find('table', {'id': 'champions_index'})
    
    yearly_teams = {}
    # Iterate through the rows of the table in tbody
    for row in table.tbody.find_all('tr'):
        # Find the link to the playoff series in data-stat="year_id"
        year = row.find('th', {'data-stat': 'year_id'})
        if year:
            # Find the league in data-stat="lg_id"
            league = row.find('td', {'data-stat': 'lg_id'}).text.strip()
            # Only consider NBA finals
            if league != 'NBA':
                print(f"Skipping league: {league}")
                continue
            # Get the year text
            year_text = year.text.strip()
            # Find the link to the playoff series
            link = row.find('a')
            if link:
                year_url = "https://www.basketball-reference.com" + link['href']
                teams = get_finals_teams(year_url, session)
                yearly_teams[year_text] = teams
    return yearly_teams

## Find teams per year

`yearly_teams` will be a list of the all of the years of Finals matchups, where each year has a link for the two teams playing

In [17]:
if SCRAPE_FINALS_YEARS:
    # Get the yearly finals teams
    yearly_teams = get_playoff_series()

    # Save dictionary to a file
    with open('yearly_finals_teams.json', 'w') as f:
        json.dump(yearly_teams, f, indent=4)

Skipping league: ABA
Skipping league: ABA
Skipping league: ABA
Skipping league: ABA
Skipping league: ABA
Skipping league: ABA
Skipping league: ABA
Skipping league: ABA
Skipping league: ABA
Skipping league: BAA
Skipping league: BAA
Skipping league: BAA


In [8]:
if not SCRAPE_FINALS_YEARS:
    # Load the saved dictionary from the file
    with open('yearly_finals_teams.json', 'r') as f:
        yearly_teams = json.load(f)

In [19]:
"""
Function to get the roster of a team from a given URL.
:param url: The URL of the team's roster page.
:return: A list of dictionaries containing player names and their URLs.
"""
def get_roster(url):
    # Request the page and parse it with BeautifulSoup
    soup = scrape("https://www.basketball-reference.com" + url)
    
    # Initialize an empty list to hold the roster
    roster = []
    
    # Find the table with the roster
    roster_table = soup.find('table', {'id': 'roster'}).tbody
    
    # Iterate through the rows of the table
    for row in roster_table.find_all('tr'):
        # Find the player name in the row
        player = row.find('td', {'data-stat': 'player'})
        if player:
            player_url = player.find('a')['href']  # This will give the player's page URL
            player_name = player.text.strip() # This will give the player's name
            roster.append({'name': player_name, 'player_url': player_url, 'team_url': url})
    
    return roster

## Find rosters for every finals team

`team_rosters` will be a list of every player that was on a finals team for every year

In [20]:
if SCRAPE_TEAM_ROSTERS:
    team_rosters = []

    for year, teams in yearly_teams.items():
        for team in teams:
            roster = get_roster(team)
            team_rosters.extend(roster)
    # Save the rosters to a file
    with open('team_rosters.json', 'w') as f:
        json.dump(team_rosters, f, indent=4)

In [9]:
if not SCRAPE_TEAM_ROSTERS:
    # Load the saved rosters from the file
    with open('team_rosters.json', 'r') as f:
        team_rosters = json.load(f)

In [21]:
rosters_df = pl.DataFrame(team_rosters)
rosters_df

name,player_url,team_url
str,str,str
"""Shai Gilgeous-Alexander""","""/players/g/gilgesh01.html""","""/teams/OKC/2025.html"""
"""Aaron Wiggins""","""/players/w/wiggiaa01.html""","""/teams/OKC/2025.html"""
"""Isaiah Joe""","""/players/j/joeis01.html""","""/teams/OKC/2025.html"""
"""Luguentz Dort""","""/players/d/dortlu01.html""","""/teams/OKC/2025.html"""
"""Jalen Williams""","""/players/w/willija06.html""","""/teams/OKC/2025.html"""
…,…,…
"""Johnny Macknowski""","""/players/m/macknjo01.html""","""/teams/SYR/1950.html"""
"""Ed Peterson""","""/players/p/petered01.html""","""/teams/SYR/1950.html"""
"""George Ratkovicz""","""/players/r/ratkoge01.html""","""/teams/SYR/1950.html"""


In [22]:
# Double check that the number of unique teams matches the expected count
len(rosters_df.group_by("team_url").len()), len(yearly_teams)*2

(152, 152)

## Find the teams that every finals player played for

`all_player_teams` will be a list of every player from `team_rosters` and all of the teams that they've played for in their career.

In [23]:
all_player_teams = []

In [26]:
# Get the unique players so we don't scrape the same player multiple times
unique_players = list(rosters_df["player_url"].unique())

if SCRAPE_PLAYER_TEAMS:
    # Check if we have already scraped some players
    # Useful if we have to restart the script
    if all_player_teams != []:
        checked_players = [player['player_url'] for player in all_player_teams]
    else:
        checked_players = []

    # Iterate through the unique players and get their teams
    for player in tqdm(unique_players):
        player_url = "https://www.basketball-reference.com" + player
        # Only scrape players that haven't been checked yet
        if player not in checked_players:
            try:
                player_teams = get_player_teams(player_url)
            except Exception as e:
                # Skip players that cause errors
                player_teams = []
            all_player_teams.append({'player_url': player, 'teams': player_teams})

    # Save the player teams to a file
    with open('player_teams.json', 'w') as f:
        json.dump(all_player_teams, f, indent=4)


100%|██████████| 1274/1274 [00:52<00:00, 24.14it/s]


In [ ]:
if not SCRAPE_PLAYER_TEAMS:
    # Read the player teams from the file
    with open('player_teams.json', 'r') as f:
        all_player_teams = json.load(f)

In [27]:
# Make sure we got every player
len(all_player_teams), len(unique_players)

(1274, 1274)

## Convert things to DataFrames

Convert data to DataFrames and clean them for joining.

In [29]:
# Convert the list of dictionaries to a Polars DataFrame
player_team_df = pl.DataFrame(all_player_teams)
player_team_df

player_url,teams
str,list[str]
"""/players/o/odomla01.html""","[""LAC"", ""LAL"", … ""MIA""]"
"""/players/p/paddige01.html""","[""WSB"", ""NYK"", … ""CLE""]"
"""/players/e/eganjo01.html""","[""DET"", ""BAL"", … ""CLE""]"
"""/players/p/parisro01.html""","[""CHI"", ""BOS"", … ""CHH""]"
"""/players/h/horanjo01.html""","[""2TM"", ""FTW"", ""MNL""]"
…,…
"""/players/i/iguodan01.html""","[""MIA"", ""GSW"", … ""DEN""]"
"""/players/h/hetzefr01.html""","[""CIN"", ""2TM"", … ""LAL""]"
"""/players/c/claxtsp01.html""","[""NOH"", ""Did not play - injury"", … ""GSW""]"


In [30]:
def filter_non_teams(x):
    # Filter out entries that are not 3-letter team abbreviations
    return [team for team in x if len(team) == 3 and team.isalpha()]

# Apply the filter function to the 'teams' column
player_team_df = player_team_df.with_columns(
    pl.col('teams').map_elements(filter_non_teams).alias('clean_teams')
).filter(
    pl.col("clean_teams").list.len() > 0 # Remove any players that didn't have any valid teams
)
player_team_df

/var/folders/cb/4zxwl0213zqd2q8g2v7nx6tm0000gn/T/ipykernel_80748/3068046824.py:6: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  player_team_df = player_team_df.with_columns(


player_url,teams,clean_teams
str,list[str],list[str]
"""/players/o/odomla01.html""","[""LAC"", ""LAL"", … ""MIA""]","[""LAC"", ""LAL"", … ""MIA""]"
"""/players/p/paddige01.html""","[""WSB"", ""NYK"", … ""CLE""]","[""WSB"", ""NYK"", … ""CLE""]"
"""/players/e/eganjo01.html""","[""DET"", ""BAL"", … ""CLE""]","[""DET"", ""BAL"", … ""CLE""]"
"""/players/p/parisro01.html""","[""CHI"", ""BOS"", … ""CHH""]","[""CHI"", ""BOS"", … ""CHH""]"
"""/players/h/horanjo01.html""","[""2TM"", ""FTW"", ""MNL""]","[""FTW"", ""MNL""]"
…,…,…
"""/players/i/iguodan01.html""","[""MIA"", ""GSW"", … ""DEN""]","[""MIA"", ""GSW"", … ""DEN""]"
"""/players/h/hetzefr01.html""","[""CIN"", ""2TM"", … ""LAL""]","[""CIN"", ""MIL"", … ""LAL""]"
"""/players/c/claxtsp01.html""","[""NOH"", ""Did not play - injury"", … ""GSW""]","[""NOH"", ""ATL"", … ""GSW""]"


In [31]:
# Reshape yearly_teams to make it easier to work with
yearly_teams_list = [[key, value[0], value[1]] for key, value in yearly_teams.items()]
# Create a Polars DataFrame from the list
yearly_teams_df = pl.DataFrame(yearly_teams_list, schema=["year", "team1", "team2"], orient= "row")
yearly_teams_df

year,team1,team2
str,str,str
"""2025""","""/teams/OKC/2025.html""","""/teams/IND/2025.html"""
"""2024""","""/teams/BOS/2024.html""","""/teams/DAL/2024.html"""
"""2023""","""/teams/DEN/2023.html""","""/teams/MIA/2023.html"""
"""2022""","""/teams/GSW/2022.html""","""/teams/BOS/2022.html"""
"""2021""","""/teams/MIL/2021.html""","""/teams/PHO/2021.html"""
…,…,…
"""1954""","""/teams/MNL/1954.html""","""/teams/SYR/1954.html"""
"""1953""","""/teams/MNL/1953.html""","""/teams/NYK/1953.html"""
"""1952""","""/teams/MNL/1952.html""","""/teams/NYK/1952.html"""


## Join

For every team that was in the finals, find all the players on their team. Then, for every player that was on a finals team, find all the teams they played for.

In [32]:
# Stack the yearly teams DataFrame to have one row per team
stacked_yearly_teams_df = pl.concat([
    yearly_teams_df.select(
        pl.col("year"),
        pl.col("team1").alias("team")
    ),
    yearly_teams_df.select(
        pl.col("year"),
        pl.col("team2").alias("team"),
    )
])

# Join the stacked yearly teams DataFrame with the roster df
teams_with_rosters = stacked_yearly_teams_df.join(
    rosters_df,
    left_on="team",
    right_on="team_url",
    how="inner"
)

# Join the teams with rosters DataFrame with the player teams DataFrame
years_with_teams = teams_with_rosters.join(
    player_team_df,
    left_on="player_url",
    right_on="player_url",
    how="inner"
)

# Get a list of unique teams for each year
teams_per_year = years_with_teams.explode("clean_teams").group_by("year").agg(
    pl.col("clean_teams").unique()
)

## Pivot

Pivot the final DataFrame so that we have a column for each team, and each row is a year. The column for each team indicates whether a player that every played for that team was in the finals that year.

In [33]:
# Pivot the DataFrame to have years as rows and teams as columns
pivoted = teams_per_year.explode("clean_teams").pivot(
    index="year",
    on="clean_teams",
    values="clean_teams",
    aggregate_function="len"
).fill_null(0).sort("year", descending=True)

In [34]:
pivoted

year,WSB,DET,CAP,FLO,BOS,PHI,BUF,BAL,KCK,MIL,NYK,SDR,HOU,SEA,POR,CLE,DLC,INA,STL,NYA,SFW,CIN,PHO,LAL,CHI,TRI,MNL,PHW,STB,WSC,CHS,FTW,ROC,MLH,SYR,BLB,…,INO,OKC,CHO,BRK,NOH,CHA,UTA,MEM,NOP,NYN,DNR,VIR,NOJ,SSL,SDA,SDS,DNA,SDC,KCO,KEN,NOK,MMT,CAR,SAA,VAN,LAS,PTP,OAK,UTS,MMP,WSA,NOB,ANA,TEX,MNP,HSM,PTC
str,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,…,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
"""2025""",0,0,0,0,1,1,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,…,0,1,0,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"""2024""",0,1,0,0,1,1,0,0,0,1,1,0,0,0,1,1,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,…,0,1,1,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"""2023""",0,1,0,0,1,1,0,0,0,1,1,0,1,1,1,1,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,…,0,1,1,1,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"""2022""",0,1,0,0,1,1,0,0,0,1,1,0,1,0,1,1,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,…,0,1,1,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"""2021""",0,1,0,0,1,1,0,0,0,1,1,0,1,0,1,1,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,…,0,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""1954""",0,1,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,1,0,1,0,1,1,1,1,1,…,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"""1953""",0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,1,1,1,1,1,1,1,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"""1952""",0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,1,0,0,1,1,1,1,1,1,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Formatting

Add a totals row and sort the columns based on the totals.

In [35]:
# Make a new row that is the sum of all columns
pivoted_sorted = pl.concat([
    pivoted,
    pivoted.select(cs.numeric()).sum().insert_column(
        0,
        pl.lit("Total").alias("year")
    )
])

In [36]:
# Get the last row without the year
last_row = pivoted_sorted[-1, 1:].row(index=0, named=True)
# Sort the columns based on the values in the last row
sorted_cols = sorted(last_row.items(), key=lambda item: item[1], reverse=True)
# Get the sorted column names
sorted_col_names = [col[0] for col in sorted_cols]

# Reorder the DataFrame columns based on the sorted column names
pivoted_sorted = pivoted_sorted.select(
    pl.col("year").alias("Year"),
    *sorted_col_names
)

In [37]:
pivoted_sorted

Year,NYK,BOS,DET,LAL,PHI,MIL,CLE,PHO,CHI,ATL,HOU,POR,GSW,SEA,SAS,IND,LAC,DEN,UTA,DAL,SAC,NJN,ORL,MIA,MIN,TOR,WSB,WAS,CHH,STL,MEM,OKC,NOH,CIN,MNL,BRK,…,DLC,WSC,CHZ,VIR,SSL,KCO,KEN,CHS,CAP,INA,UTS,TRI,SDA,CHP,INO,NYN,DNA,CAR,OAK,AND,MMT,SAA,LAS,STB,INJ,SDS,PTC,FLO,PTP,WSA,ANA,TEX,WAT,MMP,NOB,MNP,HSM
str,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,…,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
"""2025""",1,1,0,1,1,0,1,0,1,0,1,0,0,0,0,1,1,1,1,1,1,0,0,1,1,1,0,1,0,0,1,1,0,0,0,1,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"""2024""",1,1,1,1,1,1,1,1,1,1,0,1,0,0,1,1,1,0,1,1,1,0,0,1,1,1,0,1,0,0,1,1,0,0,0,1,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"""2023""",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,0,1,0,0,1,1,0,0,0,1,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"""2022""",1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,0,1,1,1,1,0,1,0,0,1,1,0,0,0,1,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"""2021""",1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,0,1,1,1,0,0,1,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""1953""",1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,…,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
"""1952""",1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,…,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
"""1951""",1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,…,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [38]:
# Write to CSV
pivoted_sorted.write_csv("teams_per_year.csv")